<a href="https://colab.research.google.com/github/miguel-173/Proyecto_1/blob/main/04_modelo_con_preprocesado_llenado_de_datos_nulos_Codificaci%C3%B3n_por_pipelines_e_hiperpar%C3%A1mteros_con_gridshearcv_y_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PROYECTO IA 2024-1**


##Carga de datos


In [ ]:
#importar librerías para lectura Kaggle
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "." #ubicación

In [ ]:
!kaggle competitions download -c udea-ai4eng-20241 #acceso a la información del Kaggle

udea-ai4eng-20241.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip udea-ai4eng-20241.zip #descargar zip con data CSV

In [ ]:
df = pd.read_csv("train.csv") #lectura de datos



## 04 - Modelo con preprocesado llenado de datos nulos. Codificación por pipelines e hiperparámteros con gridshearcv




In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

target_column = 'target'

# Separar características y objetivo
X = df.drop(columns=[target_column])
y = df[target_column]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Separar características numéricas y categóricas
caracteristicas_numericas = X.select_dtypes(include=['int64', 'float64']).columns
caracteristicas_categoricas = X.select_dtypes(include=['object']).columns

# Pipelines para transformación de datos
pipeline_numerico = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Asegura la imputación para evitar errores con el scaler
    ('scaler', MinMaxScaler())  # Cambia a StandardScaler() si prefieres estandarizar
])

pipeline_categorico = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputación de valores faltantes en categóricas
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', pipeline_numerico, caracteristicas_numericas),
    ('cat', pipeline_categorico, caracteristicas_categoricas)
])

# Crear un pipeline para el modelo completo
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Definir el grid de hiperparámetros
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__bootstrap': [True, False]
}

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=model_pipeline, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Entrenar el modelo con GridSearchCV
grid_search.fit(X_train, y_train)

# Obtener los mejores parámetros y el mejor estimador
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Mejores hiperparámetros:", best_params)

# Hacer predicciones con el mejor modelo
y_pred = best_model.predict(X_test)
